In [ ]:
import os
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tabulate import tabulate
import pandas as pd
np.set_printoptions(threshold=np.inf)


In [ ]:
# Import module yang digunkaan untuk ekstrak gambar
from preprocessing.scarpping_component import extract_component_by_images

from feature_extraction.poc import POC
from feature_extraction.vektor import Vektor
from feature_extraction.quadran import Quadran

# load the face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")
# pathDatasetImage = "dataset/video_to_images"
pathDatasetImage = "dataset/sub01/EP02_01f"
blockSize = 7
# Inisialisasi variabel untuk menampung data pertama, index, dan data csv
data_blocks_first_image = None
index = 0
frames_data = []
# looping semua file yang ada didalam
for filename in os.listdir(pathDatasetImage):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        # Create the directory if it doesn't exist
        image = cv2.imread(os.path.join(pathDatasetImage, filename))
        image = cv2.resize(image, (600, 500))
        # image = cv2.resize(image, (98, 56))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect the faces
        rects = detector(gray)

        # Inisialisasi data per frame
        frame_data = {'Frame': f"{index + 1}({filename.split('.')[0]})"}

        # Inisialisasi data untuk setiap blok
        for i in range(120):  # Jumlah blok maksimum 120
            frame_data[f'X{i+1}'] = 0
            frame_data[f'Y{i+1}'] = 0
            frame_data[f'Tetha{i+1}'] = 0
            frame_data[f'Magnitude{i+1}'] = 0

        # go through the face bounding boxes
        for rect in rects:
            # apply the shape predictor to the face ROI
            shape = predictor(gray, rect)

            # Mulut
            data_blocks_image_current = extract_component_by_images(
                image=image, 
                shape=shape, 
                block_size=blockSize,
                frameName=filename.split(".")[0],
                objectName="mouth", 
                objectRectangle={"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57}, 
                pixelShifting={"pixel_x": 25, "pixel_y": 5}
            )

            # print(data_blocks_image_current)

            if data_blocks_first_image is None:
                # Inisialisasi data pertama
                data_blocks_first_image = data_blocks_image_current
                continue

            plt.imshow(np.uint8(data_blocks_image_current), cmap="gray")

            # Inisiasi class POC
            initPOC = POC(data_blocks_first_image, data_blocks_image_current, blockSize) 
            # Pemanggilan fungsi pocCalc() untuk menghitung nilai POC disetiap gambar
            valPOC = initPOC.getPOC() 

            # print(valPOC)

            # Pemanggilan class dan method untuk menampilkan quiver / gambar panah
            initQuiv = Vektor(valPOC, blockSize)
            quivData = initQuiv.getVektor()

            plt.quiver(quivData[:, 0], quivData[:, 1], quivData[:, 2], quivData[:, 3], scale=1, scale_units='xy', angles='xy', color="r")    

            # num = 0
            for rect_def in valPOC[2]:
                x, y, width, height = rect_def
                
                rects = patches.Rectangle((x,y), width,height, edgecolor='r', facecolor='none') 
                plt.gca().add_patch(rects)
                
                # plt.text(x,y,f'({num})', color="blue") 
                # num += 1

            # Pemanggilan class untuk mengeluarkan nilai karakteristik vektor
            # blok ke, x,y,tetha, magnitude, dan quadran ke
            initQuadran = Quadran(quivData) 
            quadran = initQuadran.getQuadran()

            print(tabulate(quadran, headers=['Blok Ke', 'X', 'Y', 'Tetha', 'Magnitude', 'Quadran Ke']))
            plt.axis('on') 
            plt.show()     
            
            # Update data frame dengan data quadran
            for i, quad in enumerate(quadran):
                frame_data[f'X{i+1}'] = quad[1]
                frame_data[f'Y{i+1}'] = quad[2]
                frame_data[f'Tetha{i+1}'] = quad[3]
                frame_data[f'Magnitude{i+1}'] = quad[4]

        # Append data frame ke list
        frames_data.append(frame_data)
        # tambah index disetiap looping frame
        index += 1
            
# Buat DataFrame dari list data frame
df = pd.DataFrame(frames_data)

# Remove file output.csv
if os.path.exists('output.csv') and os.path.exists('output.xlsx'):
    os.remove('output.csv')
    os.remove('output.xlsx')
if os.path.exists('output.csv'):
    os.remove('output.csv')
if os.path.exists('output.xlsx'):
    os.remove('output.xlsx')  
# print(df.to_string())

# Export dataframe ke dalam file excel xlsx
df.to_excel('output.xlsx', index=False, float_format=None)

# Export DataFrame ke dalam file CSV
df.to_csv('output.csv', index=False, float_format=None)
# df.to_csv('output.csv', sep=',', float_format=None, encoding='utf-8')